In [1]:
import importlib
import sys
import torch
import pickle
import os

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')
sys.path.insert(0, '../../../../..')
sys.path.insert(0, '../../../../../..')

from model.dropout_uncertainty_enc_dec_LSTM.dropout_uncertainty_model import DropoutUncertaintyEncoderDecoderLSTM

In [2]:
#load model
file_path_model = '../../../training_variational_dropout_log_normal/Repair_shop/repair_shop_full_grad_lognorm_v1.pkl'
model = DropoutUncertaintyEncoderDecoderLSTM.load(file_path_model, dropout=0.1)

# Load the dataset
file_path_data_set = '../../../../../encoded_data/log_stand/repair_shop_all_lognormal_5_test.pkl'
bpic_17_test_dataset = torch.load(file_path_data_set, weights_only=False)

Data set categories:  ([('concept:name', 9, {'ACKNOWLEDGEMENT': 1, 'CREATE_INVOICE': 2, 'DISASSEMBLY': 3, 'EOS': 4, 'QUALITY_CONTROL': 5, 'RECEPTION': 6, 'REPAIR': 7, 'SHIPPING': 8})], [('day_in_week', 1, {}), ('seconds_in_day', 1, {}), ('case_elapsed_time', 1, {}), ('event_elapsed_time', 1, {})])
Encoder input features:  [['concept:name'], ['day_in_week', 'seconds_in_day', 'case_elapsed_time', 'event_elapsed_time']]
Decoder input+output features:  [['concept:name'], ['case_elapsed_time', 'event_elapsed_time']]


Sequence length of decoder output:  4


Cells hidden size:  128
Number of LSTM layer:  4
Dropout rate:  0.1


Encoder number of labels for each input feature (categorical, numerical):  [[9], [1, 1, 1, 1]]
Encoder indices of tensors in dataset used as input:  [[0], [0, 1, 2, 3]]
Embeddings encoder:  ModuleList(
  (0): Embedding(9, 5)
)
Total embedding feature size encoder:  5
Total numerical feature size encoder:  4
Input feature size encoder:  9
Encoder initialized! 

Decoder 

In [3]:
import src.evaluation.probabilistic_evaluation
importlib.reload(src.evaluation.probabilistic_evaluation)
from src.evaluation.probabilistic_evaluation import ProbabilisticEvaluation

new_eval = ProbabilisticEvaluation(model, bpic_17_test_dataset,
                                   num_processes=32,
                                   growing_num_values = ['case_elapsed_time'],
                                   samples_per_case = 1000,
                                   sample_argmax = False,
                                   use_variance_cat = True,
                                   use_variance_num = True,
                                   all_cat= ['concept:name'],
                                   all_num= ['case_elapsed_time', 'event_elapsed_time'])

In [4]:
def save_chunk(results, i):
    chunk_number = (i + 1)
    filename = os.path.join(output_dir, f'results_part_{chunk_number:03d}.pkl')
    with open(filename, 'wb') as f:
        pickle.dump(results, f)
    print(f"Saved {len(results)} results to {filename}")

output_dir = '../../../../../../../evaluation_results/repair_shop/gn_lognormal'

save_every = 50

results = {}
for i, (case_name, prefix_len, prefix, predicted_suffixes, suffix, mean_prediction) in enumerate(new_eval.evaluate_multi_processing(random_order=False, include_model_states=False)):
    # print(case_name, prefix_len)
    assert((case_name, prefix_len) not in results)
    results[(case_name, prefix_len)] = (prefix, suffix, mean_prediction, predicted_suffixes)
    # print(prefix_len, len(suffix))
    if (i + 1) % save_every == 0:
        save_chunk(results, i)
        results = {}

if len(results):
    save_chunk(results, i)

  0%|          | 0/1337 [00:00<?, ?it/s]

Saved 50 results to ../../../../../../../evaluation_results/repair_shop/gn_lognormal/results_part_050.pkl
Saved 50 results to ../../../../../../../evaluation_results/repair_shop/gn_lognormal/results_part_100.pkl
Saved 50 results to ../../../../../../../evaluation_results/repair_shop/gn_lognormal/results_part_150.pkl
Saved 50 results to ../../../../../../../evaluation_results/repair_shop/gn_lognormal/results_part_200.pkl
Saved 50 results to ../../../../../../../evaluation_results/repair_shop/gn_lognormal/results_part_250.pkl
Saved 50 results to ../../../../../../../evaluation_results/repair_shop/gn_lognormal/results_part_300.pkl
Saved 50 results to ../../../../../../../evaluation_results/repair_shop/gn_lognormal/results_part_350.pkl
Saved 50 results to ../../../../../../../evaluation_results/repair_shop/gn_lognormal/results_part_400.pkl
Saved 50 results to ../../../../../../../evaluation_results/repair_shop/gn_lognormal/results_part_450.pkl
Saved 50 results to ../../../../../../../evalu